# Prep OPERA RTC CalVal data: stage 1, part 2

**Alex Lewandowski; Alaska Satellite Facility, University of Alaska Fairbanks**

## Performs additional data preparation for OPERA RTC calibration and validation

**Notebook Requires**
- directory of geotiffs processed with Prep_OPERA_RTC_CalVal_data_stage1_part1.ipynb

**Actions**
- convert incidence angle maps from radians to degrees
- Use the Copernicus global land cover data and HyP3 layover-shadow mask to create a mask of valid land cover pixels unaffected by layover or shadow
- Determine _foreslope_, _backslope_, and _flat_ areas by subtracting the ellipsoidal incidence angles from local incidence angles
- **(GAMMA RTC)** create 3 backscatter geotiffs for each polarity containing valid pixels of the selected ground cover classification in:
    - foreslope regions
    - backslope regions
    - flat regions
- **(OPERA RTC)** create 1 backscatter geotiffs for each polarity

In [ ]:
from ipyfilechooser import FileChooser
import math
import numpy as np
from pathlib import Path
import shutil

import numpy.ma as ma
from osgeo import gdal

import opensarlab_lib as osl

In [ ]:
print("Select the *_prepped directory output by Prep_OPERA_RTC_CalVal_data_stage1_part1.ipynb")
fc = FileChooser(Path.cwd())
display(fc)

## Warp all products to pixel size of 30

In [ ]:
data_dir = Path(fc.selected_path)
tiffs = list(data_dir.glob('*.tif'))
resolution = 30
for t in tiffs:
    gdal.Warp(str(t), str(t), xRes=resolution, yRes=resolution, 
              targetAlignedPixels=True, dstNodata=None, copyMetadata=True)   

## Define GAMMA vs OPERA RTC

In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout

processor =  widgets.RadioButtons(
        options=['OPERA', 'HyP3 GAMMA'],
        description='',
        disabled=False,
        layout=Layout(min_width="800px")
    )
display(processor)

In [ ]:
opera = processor.value == 'OPERA'

## Glob product paths

In [ ]:
if opera:
    lc = '*_local_incidence_angle_*.tif'
    ls = '*_layover_shadow_mask_*.tif'
else:
    # local_inc = list(data_dir.glob('*_lc_inc_map_*.tif'))[0]
    # ls_map = list(data_dir.glob('*_ls_map_*.tif'))[0]
    lc = '*_lc_inc_map_*.tif'
    ls = '*_ls_map_*.tif'
    ellipse_inc = list(data_dir.glob('*_ell_inc_map_*.tif'))[0]

local_inc = list(data_dir.glob(f'{lc}'))[0]
ls_map = list(data_dir.glob(f'{ls}'))[0]
vh = list(data_dir.glob("*_VH_*.tif"))[0]
vv = list(data_dir.glob("*_VV_*.tif"))[0]
dem = list(data_dir.glob("*_dem_*.tif"))[0]
landcover = list(data_dir.glob("*LC100_global*.tif"))[0]

---
## Mask unwanted land cover types

### Landcover Classifications
https://lcviewer.vito.be/download

- Tree cover
    - 111
    - 112
    - 114
    - 115
    - 116

## Select a ground cover type of interest

In [ ]:
print("Select a ground cover type")
ground_cover_choice = osl.select_parameter(["Tree_Cover"])
display(ground_cover_choice)

In [ ]:
ground_cover = ground_cover_choice.value
ground_covers = [0, 111, 112, 113, 114, 115, 116, 121, 122, 123, 124, 125, 126, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200]

if ground_cover == 'Tree_Cover':
    valid_covers = [111 , 112, 114, 115, 116]

invalid_covers = [c for c in ground_covers if c not in valid_covers]

ground_cover_dir = data_dir.parent/ground_cover
if not ground_cover_dir.is_dir():
    ground_cover_dir.mkdir()

## Convert incidence angle maps to degrees

In [ ]:
out = ground_cover_dir/f"{local_inc.stem}_deg.tif"    
shutil.copy(local_inc, out)
f = gdal.Open(str(out), gdal.GA_Update)

array = f.ReadAsArray()
array = array / math.pi * 180

f.GetRasterBand(1).WriteArray(array)
f.FlushCache()

local_inc = out

## mask invalid pixels (as determined from layover-shadow mask)

## **(GAMMA RTC)** Create 3 backscatter tiffs for each polarization:

- foreslope backscatter
- backslope backscatter
- flat backscatter 

## **(OPERA RTC)** Create a single backscatter tiffs for each polarization:

Remove invalid ground cover and layover/shadow pixels from all output backscatter tiffs 

## Define Land Cover

In [ ]:
valid_landcover = ground_cover_dir/f"{landcover.stem}_valid_{ground_cover}.tif"
shutil.copy(landcover, valid_landcover)

f_landcover = gdal.Open(str(valid_landcover), gdal.GA_Update)
landcover_ar = f_landcover.ReadAsArray()

## Set (In)Valid Ground Cover Pixels

In [ ]:
# Set all invalid ground cover pixels to 0
for val in invalid_covers:
    land_cover_masked = ma.masked_values(landcover_ar, val)
    landcover_ar = land_cover_masked.filled(fill_value=0)

# set all valid ground cover pixels to 1
for val in valid_covers:
    land_cover_masked = ma.masked_values(landcover_ar, val)
    landcover_ar = land_cover_masked.filled(fill_value=1)

## Set/Update All Pixels Affected by Layover or Shadow

In [ ]:
# set all pixels affected by layover or shadow to 0
f_ls_mask = gdal.Open(str(ls_map))
ls_array = f_ls_mask.ReadAsArray()

if opera:
    ls_array[ls_array == 0] = 99
    ls_mask = ma.masked_where(ls_array==99, np.zeros(ls_array.shape))
    ls_array = ls_mask.filled(fill_value=1)   
else:
    ls_mask = ma.masked_where(ls_array!=1, ls_array)
    ls_array = ls_mask.filled(fill_value=0)

# Update valid land cover mask with invalid pixels from layover shadow mask
ls_mask = ma.masked_where(ls_array==0, landcover_ar)
landcover_ar = ls_mask.filled(fill_value=0)

# Save valid land cover mask, containing values=1 for valid land cover pixels unaffected by layover and shadow
f_landcover.GetRasterBand(1).WriteArray(landcover_ar)
f_landcover.FlushCache()

## Generate Backscatter Tiffs

In [ ]:
for pol in [vh, vv]:
    
    backscatter = ground_cover_dir/f"{pol.stem}_masked_backscatter.tif"
    shutil.copy(pol, backscatter)
    f = gdal.Open(str(backscatter), gdal.GA_Update)
    pol_array = f.ReadAsArray()

    mask = ma.masked_where(landcover_ar==0, pol_array)
    masked_array = mask.filled(fill_value=np.nan)
        
    f.GetRasterBand(1).WriteArray(masked_array)
    f.FlushCache()

*Prep_OPERA_RTC_CalVal_data_stage1_part2 - Version 2.0.1 - April 2023*

*Changes:*

- *swap stage1_part2 and stage1_part3 to finish preprocessing before MGRS tiling*